In [ ]:
!python --version

In [ ]:
!sudo apt install python3.9
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 1
!sudo update-alternatives --config python3

In [42]:
import os
import random
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
from torchvision import transforms
import matplotlib.pyplot as plt

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [161]:
# Hyper-parameters 
input_size = 62500 # 250x250 RGB inage size
num_classes = 1
neurons_per_layer=250
epochs = 5
batch_size = 25
learning_rate = 0.001

In [162]:
class FNN(nn.Module):
    def __init__(self, n_inputs, n_neurons, num_classes):
        super(FNN, self).__init__()

        self.activation = nn.Sigmoid()
        self.linear_layer_stack = nn.Sequential(
            # input layer
            nn.Linear(n_inputs, n_neurons),
            nn.ReLU(),

            # hidden layer
            nn.Linear(n_neurons, n_neurons),
            nn.ReLU(),
            nn.Linear(n_neurons, n_neurons),
            nn.ReLU(),
            nn.Linear(n_neurons, n_neurons),

            # output layer
            nn.ReLU(),
            nn.Linear(n_neurons, num_classes)
        )

    def forward(self, input):
      output = self.linear_layer_stack(input)
      return self.activation(output)


model = FNN(input_size, neurons_per_layer, num_classes)
model.to(device)

FNN(
  (activation): Sigmoid()
  (linear_layer_stack): Sequential(
    (0): Linear(in_features=62500, out_features=250, bias=True)
    (1): ReLU()
    (2): Linear(in_features=250, out_features=250, bias=True)
    (3): ReLU()
    (4): Linear(in_features=250, out_features=250, bias=True)
    (5): ReLU()
    (6): Linear(in_features=250, out_features=250, bias=True)
    (7): ReLU()
    (8): Linear(in_features=250, out_features=1, bias=True)
  )
)

In [163]:
# Loss and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [165]:
class WildFireDataset(Dataset):
  train_mode = True

  def __init__(self, custom_dataset, transforms=None, shuffle=False, test_size=0):
    self.flatten = nn.Flatten()
    self.train_dataset = self.process_dataset(custom_dataset)

    self.transforms = transforms() if transforms is not None else None

    if shuffle:
      random.shuffle(self.train_dataset)

    if test_size > 0:
      if test_size > 1:
        raise ValueError("Error: Test must be between 0-1.\nRepresenting 0% to 100%")
      test_len = int(len(self.train_dataset) * test_size)
      self.test_dataset = self.train_dataset[:test_len]
      self.train_dataset = self.train_dataset[test_len:]

  def __len__(self):
    return len(self.train_dataset) if self.train_mode else len(self.test_dataset)

  def process_dataset(self, dataset):
    custom_dataset = []
    for data in dataset:
      image = data[0]
      image = self.flatten(image)
      image = image[0]
      image = image.to(torch.float)
      label = torch.tensor(data[1], dtype=torch.float)
      custom_dataset.append((image, label))
    return custom_dataset
  
  def __getitem__(self, index):
    image, label = self.train_dataset[index] if self.train_mode else self.test_dataset[index]
    return image, label

In [181]:
def load_dataset(path, label, size=500):
  dataset = []
  for img in [path+file for file in os.listdir(path)][:size]:
    image = read_image(img)
    dataset.append((image, [label]))
  return dataset

PATH_FIRE = "/content/drive/MyDrive/wildfire-dataset/fire/"
PATH_NO_FIRE = "/content/drive/MyDrive/wildfire-dataset/no-fire/"

DATASET = []
DATASET += load_dataset(PATH_FIRE, 1.0)
DATASET += load_dataset(PATH_NO_FIRE, 0.0)

wildfire_dataset = WildFireDataset(DATASET, shuffle=True, test_size=0.2)

In [168]:
def train(epochs, dataset):
  # Train the model
  n_total_steps = len(dataset)
  for epoch in range(epochs):
      for i, (image, labels) in enumerate(dataset):
          labels = labels.to(device)

          # Forward pass
          outputs = model(image)
          loss = criterion(outputs, labels)

          # Backward and optimize
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          
          if (i+1) % 100 == 0:
              print (f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

In [182]:
print("="*20, "Training", "="*20)
wildfire_dataset.train_mode = True
print("Dataset size:", len(wildfire_dataset))
ok = input("Train [y/n]: ").lower()
if ok == "y" or ok == "yes":
  print("Begin...")
  train(epochs, wildfire_dataset)
else:
  print("Aborted")

==================== Training ====================
Dataset size: 800
Train [y/n]: y
Begin...
Epoch [1/5], Step [100/800], Loss: 100.0000
Epoch [1/5], Step [200/800], Loss: 100.0000
Epoch [1/5], Step [300/800], Loss: 0.0000
Epoch [1/5], Step [400/800], Loss: 100.0000
Epoch [1/5], Step [500/800], Loss: 0.0000
Epoch [1/5], Step [600/800], Loss: 0.0000
Epoch [1/5], Step [700/800], Loss: 0.0000
Epoch [1/5], Step [800/800], Loss: 0.0000
Epoch [2/5], Step [100/800], Loss: 100.0000
Epoch [2/5], Step [200/800], Loss: 100.0000
Epoch [2/5], Step [300/800], Loss: 0.0000
Epoch [2/5], Step [400/800], Loss: 100.0000
Epoch [2/5], Step [500/800], Loss: 0.0000
Epoch [2/5], Step [600/800], Loss: 0.0000
Epoch [2/5], Step [700/800], Loss: 0.0000
Epoch [2/5], Step [800/800], Loss: 0.0000
Epoch [3/5], Step [100/800], Loss: 100.0000
Epoch [3/5], Step [200/800], Loss: 100.0000
Epoch [3/5], Step [300/800], Loss: 0.0000
Epoch [3/5], Step [400/800], Loss: 100.0000
Epoch [3/5], Step [500/800], Loss: 0.0000
Epoch [

In [176]:
def predict(epochs, test_dataset):
  # Test the model
  # In test phase, we don't need to compute gradients (for memory efficiency)
  with torch.no_grad():
      n_correct = 0
      n_samples = 0
      for images, labels in test_dataset:
          labels = labels.to(device)
          prediction = model(images).data
          n_samples += labels.size(0)
          n_correct += (prediction == labels)

      acc = 100.0 * (n_correct / n_samples)
      print(f'Accuracy of {len(test_dataset)} test images: {acc} %')

In [183]:
print("="*20, "Testing", "="*20)
wildfire_dataset.train_mode = False
print("Dataset size:", len(wildfire_dataset))
ok = input("Test [y/n]: ").lower()
if ok == "y" or ok == "yes":
  print("Begin...")
  predict(epochs, wildfire_dataset)
else:
  print("Aborted")

==================== Testing ====================
Dataset size: 200
Test [y/n]: y
Begin...
Accuracy of 200 test images: tensor([52.]) %
